## 1. Set up project in GCP
Using console.cloud.google.com, I set up a project called **grav-waves**. This project was automatically assigned a suffix (358320) by Google (most probably because this project exists somewhere else already on GCP).

## 2. Generate owner JSON key
Next, I created a service account and generate a JSON key with owner access to that service account (see [these instructions](https://cloud.google.com/docs/authentication/getting-started#auth-cloud-implicit-python)).

## 3. Upload JSON key to Colab
I uploaded data and the JSON key from step 2 to the `grav-waves` folder containing this and the other scripts.

Next, I ran the code below to stand-up the database.

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import numpy as np
import datetime

!pip install gwosc
!pip install gwpy
from gwosc import datasets
from gwpy.timeseries import TimeSeries
from gwpy.time import tconvert
import h5py

# !pip install rpy2==3.5.1 # problems with newer versions, still not fixed: https://stackoverflow.com/questions/74283327/conversion-py2rpy-not-defined-for-objects-of-type-class-str
# %load_ext rpy2.ipython

In [ ]:
# Mount drive - needed for GBQ creds
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path = '/content/drive/MyDrive/GitHub'

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    f'{root_path}/grav-waves/grav-waves-358320-34ebfeae2689.json', scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

## 4. Import GWOSC data & clean
Load in data and create a timestamp variable so events can be accessed based on human-readable dates.

#### A. Feature data

First, I import the feature data. This is almost like "metadata" for the event that's derived from the timeseries data. It includes the masses of the merging objects, the final mass of the merged object, date-time of the merging, GPS time of the merging (special unit of time used mainly in the gravitational wave sphere) and more.

In [ ]:
gwosc_events = pd.read_csv('https://www.gw-openscience.org/eventapi/csv/allevents/')

In [ ]:
# Split out date and time from ID column, convert date to date
gwosc_events[['merger_date', 'merger_time']] = gwosc_events.commonName.str.split('_', expand = True)
gwosc_events['merger_date'] = gwosc_events['merger_date'].str.replace('GW|GRB', '', regex = True)

gwosc_events['merger_date'] = gwosc_events['merger_date'].str.replace('blind', '991231')
gwosc_events['merger_date'] = gwosc_events['merger_date'].str.replace('151012.2', '151012')
gwosc_events['merger_date'] = pd.to_datetime(gwosc_events['merger_date'], format='%y%m%d')


<ipython-input-6-db8c01b4ace8>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  gwosc_events['merger_date'] = gwosc_events['merger_date'].str.replace('151012.2', '151012')


In [ ]:
gwosc_events

,id,commonName,version,catalog.shortName,GPS,reference,jsonurl,mass_1_source,mass_1_source_lower,mass_1_source_upper,...,far_lower,far_upper,p_astro,p_astro_lower,p_astro_upper,final_mass_source,final_mass_source_lower,final_mass_source_upper,merger_date,merger_time
0,151008-v1,151008,1,GWTC-1-marginal,1.128349e+09,https://doi.org/10.7935/82H3-HH23,https://gwosc.org/eventapi/json/GWTC-1-margina...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-08,None
1,151012.2-v1,151012.2,1,GWTC-1-marginal,1.128667e+09,https://doi.org/10.7935/82H3-HH23,https://gwosc.org/eventapi/json/GWTC-1-margina...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-12,None
2,151116-v1,151116,1,GWTC-1-marginal,1.131749e+09,https://doi.org/10.7935/82H3-HH23,https://gwosc.org/eventapi/json/GWTC-1-margina...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-16,None
3,161202-v1,161202,1,GWTC-1-marginal,1.164686e+09,https://doi.org/10.7935/82H3-HH23,https://gwosc.org/eventapi/json/GWTC-1-margina...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-02,None
4,161217-v1,161217,1,GWTC-1-marginal,1.165994e+09,https://doi.org/10.7935/82H3-HH23,https://gwosc.org/eventapi/json/GWTC-1-margina...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-17,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,GW200308_173609-v1,GW200308_173609,1,GWTC-3-confident,1.267724e+09,/GWTC-3/,https://gwosc.org/eventapi/json/GWTC-3-confide...,36.4,-9.6,11.2,...,NaN,NaN,0.85660,NaN,NaN,47.4,-7.7,11.1,2020-03-08,173609
177,GW200311_115853-v1,GW200311_115853,1,GWTC-3-confident,1.267963e+09,/GWTC-3/,https://gwosc.org/eventapi/json/GWTC-3-confide...,34.2,-3.8,6.4,...,NaN,NaN,0.99000,NaN,NaN,59.0,-3.9,4.8,2020-03-11,115853
178,GW200316_215756-v1,GW200316_215756,1,GWTC-3-confident,1.268431e+09,/GWTC-3/,https://gwosc.org/eventapi/json/GWTC-3-confide...,13.1,-2.9,10.2,...,NaN,NaN,0.99000,NaN,NaN,20.2,-1.9,7.4,2020-03-16,215756
179,GW200322_091133-v1,GW200322_091133,1,GWTC-3-confident,1.268904e+09,/GWTC-3/,https://gwosc.org/eventapi/json/GWTC-3-confide...,34.0,-18.0,48.0,...,NaN,NaN,0.61501,NaN,NaN,53.0,-26.0,38.0,2020-03-22,091133


In [ ]:
# Overwrite some random values in time column
gwosc_events['merger_time'] = gwosc_events['merger_time'].str.replace('injection', '000000')
gwosc_events['merger_time'] = gwosc_events['merger_time'].fillna('000000') # Fill in NAs with 0s

merger_tm = gwosc_events['merger_time'].str

# Create list of time objects
gw_time = list(map(datetime.time, merger_tm[:2].astype(int), merger_tm[2:4].astype(int), merger_tm[4:].astype(int)))

# Combine date and time to create timestamp
gwosc_events['merger_datetime'] = list(map(datetime.datetime.combine, gwosc_events['merger_date'], gw_time))

In [ ]:
# Keep only date-time column
gwosc_events = gwosc_events.drop(['merger_date', 'merger_time'], axis = 1)

# Rename column with invalid syntax for GCP
gwosc_events = gwosc_events.rename(columns={'catalog.shortName': 'catalog_shortName'})

In [ ]:
gwosc_events.dtypes

id                                          object
commonName                                  object
version                                      int64
catalog_shortName                           object
GPS                                        float64
reference                                   object
jsonurl                                     object
mass_1_source                              float64
mass_1_source_lower                        float64
mass_1_source_upper                        float64
mass_2_source                              float64
mass_2_source_lower                        float64
mass_2_source_upper                        float64
network_matched_filter_snr                 float64
network_matched_filter_snr_lower           float64
network_matched_filter_snr_upper           float64
luminosity_distance                        float64
luminosity_distance_lower                  float64
luminosity_distance_upper                  float64
chi_eff                        

#### B. Timeseries data

**Bi. Raw timeseries data**

Next, I import the timeseries data using the `gwosc` package. Note that, before running this chunk, I had already run the first code chunk in section 5 to create the `mergers` dataset in which to store this data (in the `mergers.timeseries` table).

In [ ]:
def create_event_dict(event_nm, obs):

  try:

    # GPS of event
    event_gps = datasets.event_gps(event_nm)

    # Fetch 30s of data for event from given observatory
    gw = np.array(TimeSeries.fetch_open_data(obs, event_gps - 15, event_gps + 15))

    # Convert GPS to human-readable dates
    event_dts = list(map(tconvert, [event_gps - 15, event_gps + 15]))

    # Create range of date-times between start and end
    capture_dttms = pd.date_range(event_dts[0], event_dts[1], periods = len(gw))

    event_dict = {'id': [event_nm]*len(gw),
                  'observatory': [obs]*len(gw),
                  'size': gw,
                  'capture_dttm': capture_dttms.round(freq = 'us')} # round to microsec for GCP compatibility

    return pd.DataFrame(event_dict)

  # Some events are blanks, so skip these
  except ValueError:
    pass

In [ ]:
# Get a list of all the GW events
gw_events = datasets.find_datasets()
invalid_events = gw_events[180:] + ['BKGW170608_16KHZ_R1'] # These aren't real events
gw_events_valid = [i for i in gw_events if i not in invalid_events]

In [ ]:
# Loop through events in batches of 10 and create event dictionaries; load into GCP
# Don't use too big of a batch or you'll get a "read-timeout" error
step_sz = 10

for i in np.arange(len(gw_events_valid), step = step_sz):

  events_batch = gw_events_valid[i:(i+step_sz)]
  obs_list = ['L1']*len(events_batch) + ['H1']*len(events_batch)

  events_df = list(map(create_event_dict, events_batch*2, obs_list))
  events_full = pd.concat(events_df)

  # BigQuery appends rows to table by default if it exists, so don't need to set write disposition
  # https://cloud.google.com/bigquery/docs/samples/bigquery-load-table-dataframe
  job_config = bigquery.LoadJobConfig(schema = [bigquery.SchemaField('id', 'STRING'),
                                                bigquery.SchemaField('observatory', 'STRING')],
                                    autodetect = False,
                                    source_format = bigquery.SourceFormat.CSV)
  
  job = client.load_table_from_dataframe(events_full, 'grav-waves-358320.mergers.timeseries', job_config = job_config)

**Bii. Cleaned timeseries data**

The data below come from the GWOSC website, from the "Strain data after noise subtraction" section of the [170817 page](https://gwosc.org/events/GW170817/).

In [ ]:
# From "after noise subtraction" section: 
hf = h5py.File('/content/drive/MyDrive/GitHub/grav-waves/data/H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5', 'r')
# list(hf.keys())
strain_170817 = np.array(hf['strain']['Strain'])
strain_170817_df = pd.DataFrame({'strain': strain_170817})
strain_170817_df.to_csv('/content/drive/MyDrive/GitHub/grav-waves/data/170817_H14096Hz.csv')

**Biii. Generated timeseries data**

The below data was provided from Rafia and contains 4,000 timeseries of BNS data.

In [ ]:
# The shape of the file is 4000 x 32769. The first 32768 numbers are time series data, and the last one is the label.
bns = np.load('/content/drive/MyDrive/GitHub/grav-waves/data/strain.npy')

In [ ]:
bns_ls = []

for i in range(4000):

  strain_dict = {}
  row_arr = bns[i][0]

  strain_dict['event_id'] = [i]*32768
  strain_dict['strain'] = row_arr[:32768]
  strain_dict['time_seq'] = list(np.arange(1, 32769))
  strain_dict['label'] = [row_arr[32768]]*32768

  bns_ls.append(strain_dict)

In [ ]:
for i in range(4000):

  bns_single_df = pd.DataFrame(bns_ls[i])

  # BigQuery appends rows to table by default if it exists, so don't need to set write disposition
  # https://cloud.google.com/bigquery/docs/samples/bigquery-load-table-dataframe
  job_config = bigquery.LoadJobConfig(autodetect = True, source_format = bigquery.SourceFormat.CSV)
  
  job = client.load_table_from_dataframe(bns_single_df, 'grav-waves-358320.mergers.bns_generated', job_config = job_config)

#### C. Event metadata

Load in metadata on the type of objects that merged (as papers/other analyses have derived). Note once again that, before running this chunk, I had already run the first code chunk in section 5 to create the `mergers` dataset in which to store this data (in the `mergers.objects` table).

In [ ]:
merger_metadata = pd.read_csv('/content/drive/MyDrive/GitHub/grav-waves/data/mergers_mass.csv')

In [ ]:
merger_metadata.head()

,id,observing_run,primary_mass,secondary_mass,final_mass,primary_object,secondary_object,final_object,merger_datetime
0,GW150914,01,36.0,31.0,63.0,BH,BH,BH,2015-09-14 00:00:00
1,GW151012,01,23.0,14.0,36.0,BH,BH,BH,2015-10-12 00:00:00
2,GW151226,01,14.0,7.7,21.0,BH,BH,BH,2015-12-26 00:00:00
3,GW170104,02,31.0,20.0,49.0,BH,BH,BH,2017-01-04 00:00:00
4,GW170608,02,11.0,7.6,18.0,BH,BH,BH,2017-06-08 00:00:00


In [ ]:
merger_metadata.dtypes

id                   object
observing_run        object
primary_mass        float64
secondary_mass      float64
final_mass          float64
primary_object       object
secondary_object     object
final_object         object
merger_datetime      object
dtype: object

In [ ]:
len(merger_metadata)

91

## 5. Push dataframes to GCP
Create a dataset called **mergers** within the project (`grav-waves-358320`). Upload the newly cleaned dataframe to the `grav-waves-358320.mergers.events` table. Both the dataset and table are persistent, so this code only needs to be run once. Note that the timeseries data is already loaded using the above code.

For help specifying the schema, see [these instructions](https://cloud.google.com/bigquery/docs/schemas).

In [ ]:
# Create merger dataset
mergers = bigquery.Dataset('grav-waves-358320.mergers')
mergers.location = 'US'
mergers = client.create_dataset(mergers)

In [ ]:
# Features table
job_config = bigquery.LoadJobConfig(
    schema = [bigquery.SchemaField('id', 'STRING'),
              bigquery.SchemaField('commonName', 'STRING'),
              bigquery.SchemaField('catalog_shortName', 'STRING'),
              bigquery.SchemaField('reference', 'STRING'),
              bigquery.SchemaField('jsonurl', 'STRING')],
    autodetect = False,
    source_format = bigquery.SourceFormat.CSV)

job = client.load_table_from_dataframe(gwosc_events, 'grav-waves-358320.mergers.features', job_config = job_config)

In [ ]:
# Objects table
job_config = bigquery.LoadJobConfig(
    
    #
    # schema = [bigquery.SchemaField('id', 'STRING'),
    #           bigquery.SchemaField('observing_run', 'STRING'),
    #           bigquery.SchemaField('primary_mass', 'FLOAT64'),
    #           bigquery.SchemaField('secondary_mass', 'FLOAT64'),
    #           bigquery.SchemaField('final_mass', 'FLOAT64'),
    #           bigquery.SchemaField('primary_object', 'STRING'),
    #           bigquery.SchemaField('secondary_object', 'STRING'),
    #           bigquery.SchemaField('final_object', 'STRING'),
    #           bigquery.SchemaField('merger_datetime', 'TIMESTAMP')],
    autodetect = True,
    source_format = bigquery.SourceFormat.CSV)

job = client.load_table_from_dataframe(merger_metadata, 'grav-waves-358320.mergers.objects', job_config = job_config)

## 6. Test load
Check that the data loaded successfully. Save to a df called `gw_2015`.

In [ ]:
%%bigquery --project grav-waves-358320 gw_2015
SELECT *
FROM `mergers.features`
WHERE EXTRACT(YEAR FROM merger_datetime) = 2015;

Could not save output to variable 'gw_2015'.

ERROR:
 ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Enginemetadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7f9199fed150>)


In [ ]:
features_q = 'SELECT * FROM `grav-waves-358320.mergers.features` LIMIT 10'
gw = client.query(features_q).to_dataframe()

In [ ]:
gw.head()

,id,commonName,version,catalog_shortName,GPS,reference,jsonurl,mass_1_source,mass_1_source_lower,mass_1_source_upper,...,far,far_lower,far_upper,p_astro,p_astro_lower,p_astro_upper,final_mass_source,final_mass_source_lower,final_mass_source_upper,merger_datetime
0,GW190408_181802-v1,GW190408_181802,1,GWTC-2,1.238783e+09,/GWTC-2/,https://www.gw-openscience.org/eventapi/json/G...,24.6,-3.4,5.1,...,0.000010,None,None,1.000000,None,None,41.1,-2.8,3.9,2019-04-08 18:18:02
1,GW190412-v3,GW190412,3,GWTC-2,1.239082e+09,/GWTC-2/,https://www.gw-openscience.org/eventapi/json/G...,30.1,-5.1,4.7,...,0.000010,None,None,1.000000,None,None,37.3,-3.8,3.9,2019-04-12 00:00:00
2,GW190413_052954-v1,GW190413_052954,1,GWTC-2,1.239169e+09,/GWTC-2/,https://www.gw-openscience.org/eventapi/json/G...,34.7,-8.1,12.6,...,0.071683,None,None,0.977790,None,None,56.0,-9.2,12.5,2019-04-13 05:29:54
3,GW190413_134308-v1,GW190413_134308,1,GWTC-2,1.239198e+09,/GWTC-2/,https://www.gw-openscience.org/eventapi/json/G...,47.5,-10.7,13.5,...,0.043657,None,None,0.984900,None,None,75.5,-11.4,16.4,2019-04-13 13:43:08
4,GW190421_213856-v1,GW190421_213856,1,GWTC-2,1.239918e+09,/GWTC-2/,https://www.gw-openscience.org/eventapi/json/G...,41.3,-6.9,10.4,...,0.000774,None,None,0.999768,None,None,69.7,-8.7,12.5,2019-04-21 21:38:56


## 7. Add metadata

In [ ]:
dataset = client.get_dataset('mergers')  # Make an API request.
dataset.description = 'This dataset contains data on gravitational wave merger detections made since 2015.'
dataset = client.update_dataset(dataset, ["description"])  # Make an API request.

In [ ]:
# Dataset description
dataset = client.get_dataset('mergers')
dataset.description

'This dataset contains data on gravitational wave merger detections made since 2015.'

In [ ]:
# Tables in mergers dataset
merger_tables = list(client.list_tables('mergers'))
merger_tbl = [i.table_id for i in merger_tables]
merger_tbl

['features', 'timeseries']

In [ ]:
# Descriptions for tables
table = client.get_table('grav-waves-358320.mergers.features')

In [ ]:
table.schema

[SchemaField('id', 'STRING', 'NULLABLE', None, ()),
 SchemaField('commonName', 'STRING', 'NULLABLE', None, ()),
 SchemaField('version', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('catalog_shortName', 'STRING', 'NULLABLE', None, ()),
 SchemaField('GPS', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('reference', 'STRING', 'NULLABLE', None, ()),
 SchemaField('jsonurl', 'STRING', 'NULLABLE', None, ()),
 SchemaField('mass_1_source', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('mass_1_source_lower', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('mass_1_source_upper', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('mass_2_source', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('mass_2_source_lower', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('mass_2_source_upper', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('network_matched_filter_snr', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('network_matched_filter_snr_lower', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('network_matched_filter_snr_up

In [ ]:
# Update the features table description
table.description = '''This table contains mass data for gravitational wave merger detections made since 2015.
Of note:

"(i) GW191219_163120 and GW200115_042309: two mergers between possible neutron star-black hole pairs; the neutron star in GW191219_163120 is one of the least massive ever observed;
(ii) GW200210_092254: a merger between a black hole and an object which could either be a light black hole or a heavy neutron star;
(iii) GW200220_061928: a massive pair of black holes orbiting each other, with a combined mass 145 times heavier than the Sun;
(iv) GW191204_171526: a pair of black holes orbiting each other, in which at least one of the pair is spinning upright;
(v) GW191109_010717: a pair of black holes orbiting each other which have a combined mass 112 times heavier than the Sun, which seems to be spinning upside-down;
(vi) GW191129_134029: a ‘light’ pair of black holes that together weigh only 18 times the mass of the Sun."

Source: ARC Centre of Excellence for Gravitational Wave Discovery, LIGO/VIRGO/KAGRA
Datasource: https://www.gw-openscience.org/eventapi/html/allevents/
Information on fields in dataset: https://www.gw-openscience.org/eventportaldocs/'''

table = client.update_table(table, ["description"])

In [ ]:
# Check that the description was updated
table = client.get_table('grav-waves-358320.mergers.features')
print(table.description)

This table contains mass data for gravitational wave merger detections made since 2015.
Of note:

"(i) GW191219_163120 and GW200115_042309: two mergers between possible neutron star-black hole pairs; the neutron star in GW191219_163120 is one of the least massive ever observed;
(ii) GW200210_092254: a merger between a black hole and an object which could either be a light black hole or a heavy neutron star;
(iii) GW200220_061928: a massive pair of black holes orbiting each other, with a combined mass 145 times heavier than the Sun;
(iv) GW191204_171526: a pair of black holes orbiting each other, in which at least one of the pair is spinning upright;
(v) GW191109_010717: a pair of black holes orbiting each other which have a combined mass 112 times heavier than the Sun, which seems to be spinning upside-down;
(vi) GW191129_134029: a ‘light’ pair of black holes that together weigh only 18 times the mass of the Sun."

Source: ARC Centre of Excellence for Gravitational Wave Discovery, LIGO

In [ ]:
# Descriptions for timeseries table
ts_tbl = client.get_table('grav-waves-358320.mergers.timeseries')

In [ ]:
ts_tbl.schema

[SchemaField('id', 'STRING', 'NULLABLE', None, ()),
 SchemaField('observatory', 'STRING', 'NULLABLE', None, ()),
 SchemaField('size', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('capture_dttm', 'DATETIME', 'NULLABLE', None, ())]

In [ ]:
# Update the features table description
ts_tbl.description = '''This table contains the calibrated strain data for gravitational wave merger detections made since 2015.

Per the gwpy docs:
"Gravitational-wave detectors are time-domain instruments, recording gravitational-wave amplitude as a differential change in the lengths of each of the interferometer arms. The primary output of these detectors is a single time-stream of gravitational-wave strain (also referred to as h(t)).

Alongside the strain data, thousands of instrumental control and error signals and environmental monitors are recorded in real-time and archived for off-line study. These data are colloquially called the auxiliary channels."

Source: https://gwpy.github.io/docs/latest/timeseries/
Datasource: https://www.gw-openscience.org/eventapi/html/allevents/'''

ts_tbl = client.update_table(ts_tbl, ["description"])